In [ ]:
from core.core_log import setup_logging, get_logger
from core.core_configuration import load_config, database_config
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from datetime import datetime, timedelta
from core.usage_util import get_data_for_plotting, init_database
from core.database import DwDDataHandler, SensorDataHandler, GoogleDataHandler, UlmDeHandler, WetterComHandler
from core.plotting import PlotData,SupportedDataFrames, draw_complete_summary

setup_logging()
load_config()
log = get_logger(__name__)
auth = database_config()

init_database(SensorDataHandler, auth, SupportedDataFrames.Main.table_name)

In [ ]:
ulmde_df = get_data_for_plotting(auth, UlmDeHandler, SupportedDataFrames.ULM_DE)
ulmde_df

In [ ]:
dwd_df = get_data_for_plotting(auth, DwDDataHandler, SupportedDataFrames.DWD_DE)
dwd_df

In [ ]:
wettercom_df = get_data_for_plotting(auth, WetterComHandler, SupportedDataFrames.WETTER_COM)
wettercom_df

In [ ]:
google_df = get_data_for_plotting(auth, GoogleDataHandler, SupportedDataFrames.GOOGLE_COM)
google_df

In [ ]:
df = get_data_for_plotting(auth, SensorDataHandler, SupportedDataFrames.Main)
df

In [ ]:
# measurement data collection started on 12th august 2023, 11 hours 32 minutes 54 seconds
assert df['timestamp'].min() == pd.to_datetime("2023-08-12 11:32:54")

In [ ]:
df[["humidity", "room_temp", "cpu_temp"]].describe()

In [ ]:
corr_matrix = df[["humidity", "room_temp", "cpu_temp"]].corr()
#corr_matrix = df[["humidity", "room_temp"]].corr()
corr_matrix

In [ ]:
test =[PlotData(SupportedDataFrames.Main, df, True),
       PlotData(SupportedDataFrames.DWD_DE, dwd_df),
        PlotData(SupportedDataFrames.GOOGLE_COM, google_df),
         PlotData(SupportedDataFrames.WETTER_COM, wettercom_df),
          PlotData(SupportedDataFrames.ULM_DE, ulmde_df)]
from core.plotting import DefaultPlotCategory
merged = DefaultPlotCategory._merge_temperature_by_timestamp(DefaultPlotCategory._get_main(test), test)
draw_complete_summary(test)

In [ ]:
def adjust_x_labels(ax, df, time_ranges):
    """Adjusts X-axis labels for a given range, starting from the correct day of the time range."""
    # Get the start date of the first time range (used to calculate relative days)
    start_date = time_ranges[0][0]  # Using the first start date from time_ranges
    
    # Create a list of labels for each day in the time range
    tick_labels = [
        (start_date + pd.Timedelta(days=i)).strftime('%m-%d')
        for i in range((time_ranges[0][1] - time_ranges[0][0]).days + 1)  # Number of days in the first time range
    ]

    # Set X-ticks and corresponding labels
    ax.set_xticks(range(len(tick_labels)))  # Set tick positions
    ax.set_xticklabels(tick_labels, rotation=45, ha='right')

def compare_temperature_ranges(df: pd.DataFrame, time_ranges: list):
    """
    Compare temperature trends for multiple specified time ranges
    """
    # Define the temperature metrics to plot
    metrics = ["outside_mean", "outside_max", "outside_min"]
    titles = ["Mean Temperature", "Max Temperature", "Min Temperature"]

    fig, axes = plt.subplots(3, 1, figsize=(12, 8))
    axes = axes.flatten() 

    for i, metric in enumerate(metrics):
        for start_date, end_date in time_ranges:
            df_range = df[(df['timestamp'] >= start_date) & (df['timestamp'] <= end_date)].copy()

            # Normalize to day-of-month for comparison
            df_range['day'] = df_range['timestamp'].dt.day
            # Aggregate temperature per day
            df_grouped = df_range.groupby('day').agg({metric: 'mean'})
            axes[i].plot(df_grouped.index, df_grouped[metric], label=f"{start_date.date()} to {end_date.date()}", marker='o')


        axes[i].set_title(titles[i])
        axes[i].set_xlabel("Day of Month")
        axes[i].set_ylabel("Temperature (°C)")
        axes[i].legend()
        axes[i].grid(True)

        # Adjust x-axis labels based on time ranges
        adjust_x_labels(axes[i], df, time_ranges)

    plt.tight_layout()
    plt.show()

def get_current_day_month() -> str:
    current = datetime.now()
    month = f"0{current.month}"if len(str(current.month)) == 1 else str(current.month)
    day = f"0{current.day}"if len(str(current.day)) == 1 else str(current.day)
    return f"{month}-{day}"


import statsmodels.api as sm
from core.database import TIME_FORMAT

def plot_heatmap(ax, df_list, labels):
    """Plots a heatmap of temperature differences between time ranges."""
    data = {}

    # Collect data for all time ranges
    for _df, label in zip(df_list, labels):
        df = _df.copy()
        # Calculate the day relative to the start of the time range
        df['doy'] = (df['timestamp'] - df['timestamp'].min()).dt.days  # Day relative to the start of the range
        daily_means = df.groupby('doy')['outside_mean'].mean()
        data[label] = daily_means

    # Create DataFrame from the time ranges
    df_comp = pd.concat(data, axis=1)

    # Remove days where any of the time ranges has missing data
    df_comp = df_comp.dropna(axis=0, how='any')  # Drop rows where any column has NaN

    # Subtract the baseline (first column) from all other columns
    df_diff = df_comp.iloc[:, 1:].sub(df_comp.iloc[:, 0], axis=0)  # Difference from baseline

    # Plot heatmap
    if len(df_diff) > 0:
        sns.heatmap(df_diff.T, cmap="coolwarm", center=0, annot=False, linewidths=0.5, ax=ax)

    # Set titles and labels
    ax.set_title(f"Temperature Difference (Baseline: {labels[0]})")
    ax.set_xlabel("Days from Start of Time Range")
    ax.set_ylabel("Year Range")

    # Get the start date of the first time range (Sep 23)
    start_date = df_list[0]['timestamp'].min()

    # Format X-axis ticks as MM-DD (day-month) relative to the start of the first time range
    tick_labels = [
        (start_date + pd.Timedelta(days=i)).strftime('%m-%d')
        for i in range(len(df_comp.index))
    ]

    # Set the tick locations at intervals to avoid label clutter
    interval = max(1, len(tick_labels) // 10)  # Set an interval to show a reasonable number of labels (e.g., 10 ticks)
    ax.set_xticks(df_comp.index[::interval])  # Set tick locations at intervals
    ax.set_xticklabels(tick_labels[::interval], rotation=45, ha='right')  # Set the tick labels with 45-degree rotation

    # Add a color bar explanation
    cbar = ax.collections[0].colorbar
    cbar.set_label("Temperature Difference (°C)")




def plot_histogram(ax, df_list, labels):
    """Plots a histogram of temperature anomalies normalized by days per time range."""
    data = {}

    for df, label in zip(df_list, labels):
        df['doy'] = df['timestamp'].dt.strftime('%m-%d')  # Keeps MM-DD format
        daily_means = df.groupby('doy')['outside_mean'].mean()
        data[label] = daily_means

    df_comp = pd.concat(data, axis=1)
    base_temps = df_comp.iloc[:, 0]  # First time frame is baseline

    for col in df_comp.columns[1:]:  # Skip baseline
        anomalies = df_comp[col] - base_temps
        ax.hist(anomalies, bins=15, alpha=0.6, label=col, density=True)  # Normalize

    ax.axvline(0, color="black", linestyle="dashed", linewidth=1)
    ax.set_title("Histogram of Temperature Anomalies")
    ax.set_xlabel("Temperature Difference (°C)")
    ax.set_ylabel("Relative Frequency")
    ax.legend()

def plot_smoothed_trends(ax, df_list, labels):
    """Plot smoothed temperature trends given pre-filtered DataFrames."""
    
    for df, label in zip(df_list, labels):
        df = df.copy()  # Avoid modifying original data

        # Ensure timestamps and extract numeric day-of-year
        df['timestamp'] = pd.to_datetime(df['timestamp'], format=TIME_FORMAT)
        df['doy'] = df['timestamp'].dt.dayofyear

        # Handle NaNs with interpolation before smoothing
        df['outside_mean'] = df['outside_mean'].interpolate(method='linear')
        df = df.dropna(subset=['outside_mean'])  # Drop remaining NaNs

        if len(df) > 5:  # Ensure enough points for smoothing
            smoothed = sm.nonparametric.lowess(df['outside_mean'], df['doy'], frac=0.2, return_sorted=True)
            ax.plot(smoothed[:, 0], smoothed[:, 1], label=label)

    ax.set_title("Smoothed Mean Temperature Trends")
    ax.set_xlabel("Day of Year")
    ax.set_ylabel("Mean Temperature (°C)")
    ax.legend()
    ax.grid(True)
    


def extract_time_range_data(df, time_ranges):
    data = []
    for start_date, end_date in time_ranges:
        df_range = df[(df['timestamp'] >= start_date) & (df['timestamp'] <= end_date)].copy()
        data.append(df_range)

    return data

def generate_time_range_labels(time_ranges):
    """Generate labels for given time ranges in 'MMM YYYY - MMM YYYY' format."""
    labels = []
    for start_date, end_date in time_ranges:
        label = f"{start_date.strftime('%b %Y')} - {end_date.strftime('%b %Y')}"
        labels.append(label)
    return labels


def summarize_temperature_comparison(_df: pd.DataFrame, time_ranges: list=None):
    """
    Compare temperature trends using smoothed trend lines, a heatmap, and a histogram.

    :param df: DataFrame containing 'timestamp', 'outside_mean', 'outside_max', 'outside_min'.
    :param time_ranges: List of tuples [(start_date1, end_date1), (start_date2, end_date2), ...].
    """
    if time_ranges is None:
        month_day_start="11-01"
        month_day_end= get_current_day_month()
        time_ranges = [
            #(pd.Timestamp(f"2023-{month_day_start}"), pd.Timestamp(f"2024-{month_day_end}")),
            #(pd.Timestamp(f"2024-{month_day_start}"), pd.Timestamp(f"2025-{month_day_end}"))
            (pd.Timestamp(f"2023-12-01"), pd.Timestamp(f"2023-12-14")),
            (pd.Timestamp(f"2024-12-01"), pd.Timestamp(f"2024-12-14"))
        ]

    # prepare data
    df = _df.copy()
    df = df.drop('inside_temp', axis=1)
    df = df.dropna()
    df['timestamp'] = pd.to_datetime(df['timestamp'], format=TIME_FORMAT)
    df_comp = extract_time_range_data(df, time_ranges)
    labels = generate_time_range_labels(time_ranges)
    #print(time_ranges)
    print(df_comp)
    #print(labels)

    # Create figure with 3 subplots
    fig, axes = plt.subplots(1, 3, figsize=(25, 6))
    axs = axes.flatten()

    plot_smoothed_trends(axs[0], df_comp, labels)
    plot_heatmap(axs[1], df_comp, labels)
    plot_histogram(axs[2], df_comp, labels)

    plt.tight_layout()
    plt.show()

    compare_temperature_ranges(merged, time_ranges)

summarize_temperature_comparison(merged)

In [ ]:
plt.figure(figsize=(25, 10))
plt.subplot(2, 1, 1)

sns.lineplot(label="CPU", x="timestamp", y="cpu_temp", data=df)
plt.title("Temperature Over Time")
plt.xlabel("Time")
plt.ylabel("Temp (°C)")
plt.legend()
plt.xticks(rotation=45)

plt.subplot(2, 1, 2)
sns.lineplot(label="CPU", x="timestamp", y="cpu_temp",marker='o',markersize=6, data=df[df["timestamp"] >= datetime.now() - timedelta(hours=25)])
plt.title("Temperature Last 24h")
plt.xlabel("Time")
plt.ylabel("Temp (°C)")
plt.legend()
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(25, 10))
plt.subplot(2, 1, 1)
sns.lineplot(label="Google Forecast", x="timestamp", y="wind", data=google_df)
plt.title("Wind Speed Over Time")
plt.xlabel("Time")
plt.ylabel("Speed (Km/h)")
plt.legend()
plt.xticks(rotation=45)

plt.subplot(2, 1, 2)
sns.lineplot(label="Google Forecast", x="timestamp", y="precipitation", data=google_df)
plt.title("Precipitation Chance Over Time")
plt.xlabel("Time")
plt.ylabel("precipitation (%)")
plt.legend()
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
fitness = {}#pd.read_csv('fitness.csv')
#fitness['time'] = fitness['time'].map(lambda x: datetime.strptime(x, "%d-%m-%Y"))
#fitness['weight'] = fitness['weight'].map(lambda x: float(x))
#fitness['bmi'] = fitness['bmi'].map(lambda x: float(x))
#fitness['body_fat'] = fitness['body_fat'].map(lambda x: float(x))
##fitness['muscle'] = fitness['muscle'].map(lambda x: float(x))
#fitness['kcal'] = fitness['kcal'].map(lambda x: float(x))
#fitness['visceral'] = fitness['visceral'].map(lambda x: float(x))
#fitness = fitness.sort_values(by="time")

#fitness.drop(['time'], axis=1).describe()

In [ ]:
def plot_fitness(fitness):
    plt.figure(figsize=(25, 10))

    # Visualization 1: Line plot for weight over time
    plt.subplot(2, 2, 1)
    sns.lineplot(data=fitness, x='time', y='weight')
    plt.xticks(rotation=45)
    plt.title('Weight over Time')

    # Visualization 2: Line plot for BMI over time
    plt.subplot(2, 2, 2)
    sns.lineplot(data=fitness, x='time', y='bmi')
    plt.xticks(rotation=45)
    plt.title('BMI over Time')

    # Visualization 3: Line plot for body fat over time
    plt.subplot(2, 2, 3)
    sns.lineplot(data=fitness, x='time', y='body_fat')
    plt.xticks(rotation=45)
    plt.title('Body Fat over Time')

    # Visualization 4: Line plot for muscle over time
    plt.subplot(2, 2, 4)
    sns.lineplot(data=fitness, x='time', y='muscle')
    plt.xticks(rotation=45)
    plt.title('Muscle over Time')

    plt.tight_layout()
    plt.show()

def plot_fitness_corr(fitness):
    correlation_matrix = fitness[["weight", "bmi", "body_fat", "muscle", "kcal", "visceral"]].corr()
    plt.figure(figsize=(10, 8))
    mask = np.triu(np.ones_like(correlation_matrix, dtype=bool))
    sns.heatmap(correlation_matrix, annot=True,mask=mask, cmap='coolwarm', center=0)
    plt.title('Correlation Heatmap')
    plt.show()

#plot_fitness(fitness)
#plot_fitness_corr(fitness)

def plot_overall_heatmap():
    heat_df = pd.concat([
        df[df["timestamp"] >= datetime.now() - timedelta(hours=25)][["humidity", "room_temp", "cpu_temp"]].rename(lambda x: f"home_{x}", axis="columns"),
        google_df[google_df["timestamp"] >= datetime.now() - timedelta(hours=25)].drop(['id', 'timestamp'],axis=1).rename(lambda x: f"google_{x}", axis="columns"),
        dwd_df[dwd_df["timestamp"] >= datetime.now() - timedelta(hours=25)].drop(['id', 'timestamp'],axis=1).rename(lambda x: f"dwd_{x}", axis="columns")
    ])

    m = heat_df.corr()
    plt.figure(figsize=(10, 8))
    mask = np.triu(np.ones_like(m, dtype=bool))
    sns.heatmap(m, annot=True,mask=mask, cmap='coolwarm', center=0)
    plt.title('Correlation Heatmap')
    plt.show()
    return m

#plot_overall_heatmap()